# CS4248 Project Group 23

In [1]:
# If you wish to run this on Google Colab, mount the Google drive by running this cell or click the `files` icon on the left navbar
# and click mount Google Drive (it takes some time to load)
# from google.colab import drive
# drive.mount('/content/drive')

# %cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"
# !cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"

In [2]:
# To check if torch is working on m1
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

MPS device not found.


In [3]:
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

In [4]:
# Unzip raw_data.zip locally
import zipfile
with zipfile.ZipFile('raw_data.zip', 'r') as zip_ref:
    zip_ref.extractall()

Feature Engineering: Capture various features of the text (e.g. punctuation, stopwords, statement length). 
Test out different tokenizers to capture their performance.


In [5]:
import gensim.downloader as api

def load_glove_model():
    glove_model = api.load('glove-wiki-gigaword-300')
    print("Done.",len(glove_model)," words loaded!")
    return glove_model, glove_model.vector_size

glove_model, glove_dim = load_glove_model()

Done. 400000  words loaded!


In [6]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# todo parallelize this in future


def preprocess_text(text):
    # Lowercase text
    text = text.lower()
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # tokenize
    words = text.split()
    # Get the GloVe vectors
    vectors = [glove_model[word] for word in words if word in glove_model]
    # If vectors is empty, return a vector of zeros
    if not vectors:
        print("No vectors found for the text: ", text)
        return np.zeros(glove_model.vector_size)
    return np.mean(vectors, axis=0)

In [7]:
train = pd.read_csv("./raw_data/fulltrain.csv", names=['Verdict', 'Text'])
x_train = np.array([preprocess_text(text) for text in train['Text']]) 
y_train = train['Verdict'].values - 1 # subtract 1 to make the labels 0-based

No vectors found for the text:  abfs daksl 
No vectors found for the text:  nsas xkeyscore explainedyoutube 


Test out different kinds of models and find the most effective architectures.|

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [10]:
# Vectorize the text if you didn't use GloVe
from sklearn.feature_extraction.text import tfidfVectorizer
vectorizer = tfidfVectorizer()
vectorizer.fit(x_train)
x_train = vectorizer.transform(x_train).toarray()

ImportError: cannot import name 'tfidfVectorizer' from 'sklearn.feature_extraction.text' (/home/t1dus/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py)

In [ ]:
def train_model(model, X_train, y_train, num_epochs=5, learning_rate=0.001):
    # Convert numpy arrays to PyTorch tensors
    X_train = torch.from_numpy(X_train).float()
    y_train = torch.from_numpy(y_train).long()

    # Create a DataLoader for the training data
    train_data = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_data, batch_size=16)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        for i, (texts, labels) in enumerate(train_loader):
            outputs = model(texts)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [ ]:
# train your model
model = SimpleNN(input_size=300, hidden_size=100, num_classes=4) # for now assume that we are doing the 4 class labels (0, 1, 2, 3
train_model(model, x_train, y_train)

x_train_tensor = torch.from_numpy(x_train).float()
y_pred = model(x_train_tensor).argmax(dim=1).numpy()

In [ ]:
# Or load a previously saved model
model = SimpleNN(input_size=300, hidden_size=100, num_classes=4)
model.load_state_dict(torch.load('./trained_models/model.pth'))

<All keys matched successfully>

Perform hyperparameter tuning on best 3 models.

In [ ]:
from sklearn.model_selection import GridSearchCV

# or perform hyperparameter tuning
# Create the hyperparameters grid
param_grid = {

}

grid_search = GridSearchCV(())

# Train
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

# Use best model
model = grid_search.best_estimator_

In [ ]:
# get the training error
f1_score(y_train, y_pred, average='macro')
# print for the train set f1 score is
print("Train Set: " + str(f1_score(y_train, y_pred, average='macro')))

Train Set: 0.8681960038153329


In [ ]:
# get the prediction for the test set
test = pd.read_csv('./raw_data/balancedtest.csv')
X_test = np.array([preprocess_text(text) for text in test.iloc[:, 1]]) 
X_test_tensor = torch.from_numpy(X_test).float()
output = model(X_test_tensor)
_, result = torch.max(output.data, 1)

# adjust the labels in the test set to be in the range 0-3
y_test = test.iloc[:, 0].values - 1

# get the f1 score against the test set
print("Test Set: " + str(f1_score(y_test, result.numpy(), average='macro')))

TypeError: 'LogisticRegression' object is not callable

In [ ]:
# save the model
torch.save(model.state_dict(), './trained_models/model.pth')